In [1]:
import pandas as pd
import os
import shutil
from PIL import Image
from keras.models import Model
from keras.applications import VGG16, ResNet50
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle, resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
import tensorflow.keras as keras
from keras import layers
import matplotlib.pyplot as plt
from tabulate import tabulate
import joblib
from sklearn.svm import SVC
from skimage import color, io, transform
import pickle
class_label = ['Non-Default(0)','Default(1)'] # env var

In [2]:
# Load the dataset
df = pd.read_csv('D:\Barath Suresh Docs\PROGRAMMING\MACHINE LEARNING\credit_card_fraud_detection\creditcard.csv')
# Separate the fraud and non-fraud examples
fraud = df[df['Class'] == 1]
non_fraud = df[df['Class'] == 0]
# Random under-sampling of the majority class
non_fraud_downsampled = resample(non_fraud, replace=False, n_samples=len(fraud), random_state=42)
# Combine minority class with downsampled majority class
balanced_df = pd.concat([fraud, non_fraud_downsampled])
# Shuffle the examples
balanced_df = shuffle(balanced_df, random_state=42)
# Print the class distribution of the balanced dataset
print(balanced_df['Class'].value_counts())

0    492
1    492
Name: Class, dtype: int64


In [3]:
# # Create images of transaction amount against time
# for index, row in balanced_df.iterrows():
#     filename = f"datasetImages//{str(int(row['Class']))}//{index}.png"
#     img = Image.new('RGB', (224, 224), color='white')
#     pixels = img.load()
#     for i in range(224):
#         for j in range(224):
#             pixel = int(df.iloc[index, j % 28] * 255)
#             pixels[i, j] = (pixel, pixel, pixel)
#     img.save(filename)

In [4]:
# Define the path to the directory containing the PNG files
data_dir = 'D:\\Barath Suresh Docs\\PROGRAMMING\\MACHINE LEARNING\\credit_card_fraud_detection\\novelty\\datasetImages'

In [5]:
# # Split the files into training and validation sets
# train_dir = os.path.join(data_dir, "training")
# val_dir = os.path.join(data_dir, "validation")
# if not os.path.exists(train_dir):
#     os.makedirs(train_dir)
# if not os.path.exists(val_dir):
#     os.makedirs(val_dir)

# classes = ["0", "1"]  # classes corresponding to fraud and non-fraud transactions
# for cls in classes:
#     filenames = os.listdir(os.path.join(data_dir, cls))
#     train_files, val_files = train_test_split(filenames, test_size=0.2, random_state=42)
    
#     for filename in train_files:
#         src = os.path.join(data_dir, cls, filename)
#         dst = os.path.join(train_dir, cls, filename)
#         if not os.path.exists(os.path.join(train_dir, cls)):
#             os.makedirs(os.path.join(train_dir, cls))
#         shutil.copyfile(src, dst)
        
#     for filename in val_files:
#         src = os.path.join(data_dir, cls, filename)
#         dst = os.path.join(val_dir, cls, filename)
#         if not os.path.exists(os.path.join(val_dir, cls)):
#             os.makedirs(os.path.join(val_dir, cls))
#         shutil.copyfile(src, dst)

In [6]:
# # Create a list of file paths and corresponding labels
# filepaths = []
# labels = []
# print(os.listdir(data_dir))
# for filename in os.listdir(data_dir):
#     if filename.endswith('.png'):
#         label = int(''.join(filter(str.isdigit, filename)))
#         filepaths.append(os.path.join(data_dir, filename))
#         labels.append(label)
# print(filepaths)

In [7]:
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(filepaths, labels, test_size=0.2, random_state=42)

In [8]:
# Load the VGG16 model
vgg_model = VGG16(include_top=False, input_shape=(224, 224, 3))

In [10]:
# Generate image features using the VGG16 model
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(directory='datasetImages',
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='binary',
                                                    subset='training')

validation_generator = train_datagen.flow_from_directory(directory='datasetImages',
                                                         target_size=(224, 224),
                                                         batch_size=32,
                                                         class_mode='binary',
                                                         subset='validation')

train_features = vgg_model.predict(train_generator)
test_features = vgg_model.predict(validation_generator)

Found 788 images belonging to 2 classes.
Found 196 images belonging to 2 classes.
7/7 [==============================] - 20s 3s/step


In [22]:
# Train an SVM classifier on the generated image features
print(train_features.shape)
train_features = np.random.rand(788,7,7,512)
flattened_feat = train_features.reshape(788,-1)
svm_classifier = SVC(kernel="linear")
svm_classifier.fit(flattened_feat, train_generator.classes)

(788, 7, 7, 512)


SVC(kernel='linear')

In [23]:
# Evaluate the SVM classifier on the testing set
print(test_features.shape)
test_features = np.random.rand(196,7,7,512)
flattened_feat_test = test_features.reshape(196,-1)
svm_predictions = svm_classifier.predict(flattened_feat_test)
svm_accuracy = accuracy_score(validation_generator.classes, svm_predictions)
print('SVM accuracy: {:.2f}%'.format(svm_accuracy * 100))

(196, 7, 7, 512)
SVM accuracy: 80.10%
